# Feed Forward Regression


### Importing packages


In [77]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

### Loading test data


In [78]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

testing_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

### Loading data and extracting X and y for specific bidding area


In [115]:
train_data = pd.read_csv("train.csv").iloc[15000:]
test_data = pd.read_csv("test.csv").iloc[5000:]

bidding_area = 3

X_train = pd.concat(
    [
        train_data["timestamp"],
        train_data[f"NO{bidding_area}_temperature"].rename("temperature"),
        train_data["time_of_day"],
        train_data["day_of_week"],
        train_data["day_of_year"],
        train_data[f"lagged_mean_NO{bidding_area}_24h"].rename("lagged_mean_24h"),
    ],
    axis="columns",
)

y_train = pd.concat(
    [
        train_data["timestamp"],
        train_data[f"NO{bidding_area}_consumption"].rename("consumption"),
    ],
    axis="columns",
)

X_test = pd.concat(
    [
        test_data["timestamp"],
        test_data[f"NO{bidding_area}_temperature"].rename("temperature"),
        test_data["time_of_day"],
        test_data["day_of_week"],
        test_data["day_of_year"],
        test_data[f"lagged_mean_NO{bidding_area}_24h"].rename("lagged_mean_24h"),
    ],
    axis="columns",
)

y_test = pd.concat(
    [
        test_data["timestamp"],
        test_data[f"NO{bidding_area}_consumption"].rename("consumption"),
    ],
    axis="columns",
)

X_test.set_index("timestamp", inplace=True)
y_test.set_index("timestamp", inplace=True)

X_train.set_index("timestamp", inplace=True)
y_train.set_index("timestamp", inplace=True)

test_data

,timestamp,NO1_temperature,NO2_temperature,NO3_temperature,NO4_temperature,NO5_temperature,NO1_consumption,NO2_consumption,NO3_consumption,NO4_consumption,NO5_consumption
5000,2023-03-28 03:00:00+00:00,-7.2,-0.9,-11.8,-11.9,1.0,4943.633023,4561.345384,3651.531877,2793.823143,2051.904967
5001,2023-03-28 04:00:00+00:00,-7.7,-0.3,-12.3,-11.2,1.5,5452.060606,4880.827210,3816.846924,2894.494013,2184.974990
5002,2023-03-28 05:00:00+00:00,-8.7,0.0,-12.7,-11.6,0.1,5814.857136,5145.163323,3987.918663,3008.695069,2315.692348
5003,2023-03-28 06:00:00+00:00,-8.4,0.0,-12.8,-11.5,-0.1,5816.748411,5217.189132,4034.737464,2999.242974,2360.120054
5004,2023-03-28 07:00:00+00:00,-7.6,0.3,-12.2,-9.7,0.0,5654.852452,5112.209877,3977.890725,2950.354184,2370.040717
...,...,...,...,...,...,...,...,...,...,...,...
11688,2023-12-31 19:00:00+00:00,-4.7,4.1,-3.7,-6.5,6.3,5608.067736,4962.057148,3837.204332,2806.182692,2224.449164
11689,2023-12-31 20:00:00+00:00,-4.6,4.7,-3.6,-6.4,6.3,5469.722792,4864.162484,3773.021260,2764.928700,2175.957992
11690,2023-12-31 21:00:00+00:00,-4.5,4.6,-4.2,-6.4,6.2,5321.221040,4793.361440,3725.022064,2738.213476,2137.724404
11691,2023-12-31 22:00:00+00:00,-4.5,4.6,-4.2,-6.5,5.7,5222.770756,4742.807424,3668.956000,2699.056760,2110.928436


### Creating model


In [116]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


# Define model
class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Linear(5, 64)
        self.stack = nn.Sequential(
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
        )
        self.output = nn.Linear(64, 1)

    def forward(self, x):
        x = self.input(x)
        x = self.stack(x)
        x = self.output(x)
        return x


model = FeedForward().to(device)
model

Using mps device


FeedForward(
  (input): Linear(in_features=5, out_features=64, bias=True)
  (stack): Sequential(
    (0): ReLU()
    (1): Linear(in_features=64, out_features=64, bias=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=64, bias=True)
  )
  (output): Linear(in_features=64, out_features=1, bias=True)
)

### Loss function and optimizer


In [117]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

### Training


In [118]:
def train(X_train, y_train, model, loss_fn, optimizer):
    size = len(X_train)

    X_tensor = torch.tensor(X_train.values, dtype=torch.float32)
    y_tensor = torch.tensor(y_train.values, dtype=torch.float32)

    for batch, (X, y) in enumerate(zip(X_tensor, y_tensor)):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

### Testing


In [119]:
def test(X_test, y_test, model, loss_fn):
    size = len(X_test)
    num_batches = len(X_test)

    X_tensor = torch.tensor(X_test.values, dtype=torch.float32)
    y_tensor = torch.tensor(y_test.values, dtype=torch.float32)

    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in zip(X_tensor, y_tensor):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(
        f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n"
    )

### Run epochs


In [120]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(X_train, y_train, model, loss_fn, optimizer)
    test(X_test, y_test, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.534097  [    5/20054]
loss: 0.000308  [  505/20054]
loss: 0.001074  [ 1005/20054]
loss: 0.000005  [ 1505/20054]
loss: 0.000201  [ 2005/20054]
loss: 0.000007  [ 2505/20054]
loss: 0.087262  [ 3005/20054]
loss: 0.005955  [ 3505/20054]
loss: 0.008889  [ 4005/20054]
loss: 0.042492  [ 4505/20054]
loss: 0.002389  [ 5005/20054]
loss: 0.000119  [ 5505/20054]
loss: 0.010602  [ 6005/20054]
loss: 0.000151  [ 6505/20054]
loss: 0.043090  [ 7005/20054]
loss: 0.000206  [ 7505/20054]
loss: 0.013040  [ 8005/20054]
loss: 0.005018  [ 8505/20054]
loss: 0.000014  [ 9005/20054]
loss: 0.000150  [ 9505/20054]
loss: 0.005404  [10005/20054]
loss: 0.040571  [10505/20054]
loss: 0.003868  [11005/20054]
loss: 0.000521  [11505/20054]
loss: 0.000017  [12005/20054]
loss: 0.013846  [12505/20054]
loss: 0.005709  [13005/20054]
loss: 0.007155  [13505/20054]
loss: 0.000014  [14005/20054]
loss: 0.000097  [14505/20054]
loss: 0.002576  [15005/20054]
loss: 0.001039  [15505/20054]


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)